'https://github.com/aenodehi/RUL_Prediction/tree/main'

### Temporal Fusion Transformer (TFT)

In [5]:
!pip install -q darts

In [6]:
!pip install --upgrade jupyter ipywidgets

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid

from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel

In [8]:
%pwd

'/home/workspace/src'

In [9]:
# df = pd.read_csv("/home/workspace/data/B0005_discharge.csv", parse_dates = True)
# df['datetime']

In [10]:
# from datetime import timedelta
# df = pd.read_csv("/home/workspace/data/B0005_discharge.csv", parse_dates = True)

# df['datetime'] = pd.to_datetime(df['datetime'])

# adjusted_datetimes = []

# current_base_time = df.loc[0, 'datetime']
# counter = 0

# for i in range(len(df)):
#     if df.loc[i, 'datetime'] != current_base_time:
#         current_base_time = df.loc[i, 'datetime']
#         counter = 0

#     adjusted_datetimes.append(current_base_time + timedelta(seconds=15 * counter))
#     counter += 1

# df['datetime_'] = adjusted_datetimes

# print(df[['datetime', 'datetime_']].head())

In [11]:
# df.to_csv("/home/workspace/data/B0005_discharge_adjusted.csv", index=False)

In [12]:
# df.iloc[196]

In [27]:
df = pd.read_csv("/home/workspace/data/B0005_discharge_adjusted.csv", index_col = "datetime_", parse_dates = True)

df.head()

,Unnamed: 0,cycle,type,ambient_temperature,time,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,Capacity,datetime
datetime_,,,,,,,,,,,,,
2008-04-02 15:25:41,789,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000,1.856487,2008-04-02 15:25:41
2008-04-02 15:25:56,790,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781,1.856487,2008-04-02 15:25:41
2008-04-02 15:26:11,791,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703,1.856487,2008-04-02 15:25:41
2008-04-02 15:26:26,792,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781,1.856487,2008-04-02 15:25:41
2008-04-02 15:26:41,793,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922,1.856487,2008-04-02 15:25:41


In [ ]:
from datetime import timedelta

In [25]:
unique_rows = df.loc[~df.index.duplicated(keep='first')]

unique_rows

,Unnamed: 0,cycle,type,ambient_temperature,time,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,Capacity
datetime,,,,,,,,,,,,
2008-04-02 15:25:41,789,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,4.191492,-0.004902,24.330034,-0.0006,0.0,0.0,1.856487
2008-04-02 19:43:48,1926,4,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.9000e+01 4...,4.189773,0.000021,24.697752,-0.0006,0.0,0.0,1.846327
2008-04-03 00:01:06,3059,6,discharge,24,[2.008e+03 4.000e+00 3.000e+00 0.000e+00 1.000...,4.188187,-0.001754,24.734266,-0.0006,0.0,0.0,1.835349
2008-04-03 04:16:37,4187,8,discharge,24,[2008. 4. 3. 4. 16. ...,4.188461,-0.002775,24.654236,-0.0006,0.0,0.0,1.835263
2008-04-03 08:33:25,5318,10,discharge,24,[2008. 4. 3. 8. 33. ...,4.188299,-0.007981,24.524797,-0.0006,0.0,0.0,1.834646
...,...,...,...,...,...,...,...,...,...,...,...,...
2008-05-26 10:44:38,575760,600,discharge,24,[2008. 5. 26. 10. 44. ...,4.196641,-0.003172,23.741424,0.0006,0.0,0.0,1.293464
2008-05-26 15:30:43,579702,604,discharge,24,[2008. 5. 26. 15. 30. ...,4.195624,-0.005227,23.803979,0.0006,0.0,0.0,1.288003
2008-05-26 20:21:04,583737,608,discharge,24,[2008. 5. 26. 20. 21. ...,4.195862,-0.000555,23.849122,0.0006,0.0,0.0,1.287453


In [20]:
df.index.unique()

DatetimeIndex(['2008-04-02 15:25:41', '2008-04-02 19:43:48',
               '2008-04-03 00:01:06', '2008-04-03 04:16:37',
               '2008-04-03 08:33:25', '2008-04-03 12:55:10',
               '2008-04-03 17:17:16', '2008-04-03 21:28:14',
               '2008-04-04 01:38:15', '2008-04-04 05:48:08',
               ...
               '2008-05-25 10:51:37', '2008-05-25 15:37:08',
               '2008-05-25 20:23:04', '2008-05-26 01:13:02',
               '2008-05-26 06:01:08', '2008-05-26 10:44:38',
               '2008-05-26 15:30:43', '2008-05-26 20:21:04',
               '2008-05-27 15:52:41', '2008-05-27 20:45:42'],
              dtype='datetime64[ns]', name='datetime', length=168, freq=None)

In [21]:
df.info

<bound method DataFrame.info of                      Unnamed: 0  cycle       type  ambient_temperature  \
datetime                                                                 
2008-04-02 15:25:41         789      2  discharge                   24   
2008-04-02 15:25:41         790      2  discharge                   24   
2008-04-02 15:25:41         791      2  discharge                   24   
2008-04-02 15:25:41         792      2  discharge                   24   
2008-04-02 15:25:41         793      2  discharge                   24   
...                         ...    ...        ...                  ...   
2008-05-27 20:45:42      591725    614  discharge                   24   
2008-05-27 20:45:42      591726    614  discharge                   24   
2008-05-27 20:45:42      591727    614  discharge                   24   
2008-05-27 20:45:42      591728    614  discharge                   24   
2008-05-27 20:45:42      591729    614  discharge                   24   

     

In [11]:
df = pd.read_csv("/home/workspace/data/combined_discharge.csv", parse_dates = True)
df.head(10)

,Batt_name,cycle,type,ambient_temperature,time,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,Capacity
0,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,4.191492,0.004902,24.330034,0.0006,0.000,0.000,1.856487
1,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,4.190749,0.001478,24.325993,0.0006,4.206,16.781,1.856487
2,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.974871,2.012528,24.389085,1.9982,3.062,35.703,1.856487
3,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.951717,2.013979,24.544752,1.9982,3.030,53.781,1.856487
4,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.934352,2.011144,24.731385,1.9982,3.011,71.922,1.856487
5,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.920058,2.013007,24.909816,1.9982,2.991,90.094,1.856487
6,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.907904,2.014400,25.105884,1.9982,2.977,108.281,1.856487
7,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.897036,2.011603,25.317019,1.9982,2.967,126.453,1.856487
8,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.887477,2.018015,25.509423,1.9982,2.959,144.641,1.856487
9,B0005,2,discharge,24,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,3.878959,2.013135,25.703603,1.9982,2.951,162.844,1.856487
